In [12]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [13]:
!nvidia-smi

Wed Mar 26 10:03:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.70                 Driver Version: 572.70         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   42C    P3             11W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [15]:
def load_activity_data(base_dir):
    """
    Load CSV files with 2D matrices of subcarriers and packets
    
    Args:
        base_dir (str): Base directory containing 'front' and 'side' folders
    
    Returns:
        tuple: X (features), y (labels)
    """
    X = []
    y = []
    
    # Perspectives to process
    activities = ['jump', 'nopeople', 'squatting', 'standing', 'twist']
    
    for activity in activities:
        activity_path = os.path.join(base_dir, activity)
            
        # Process CSV files in each activity folder
        csv_files = [f for f in os.listdir(activity_path) if f.endswith('.csv')]
        
        for csv_file in csv_files:
            file_path = os.path.join(activity_path, csv_file)
            
            try:
                # Read CSV file as a 2D matrix
                data = pd.read_csv(file_path, header=None).values
                
                # Normalize the matrix
                data = (data - np.mean(data)) / np.std(data)
                
                X.append(data)
                y.append(activity)
            
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

    return X, y
 

In [16]:
def prepare_data(X, y, test_size=0.2):
    """
    Prepare data for model input
    
    Args:
        X (list): List of 2D matrices
        y (list): List of labels
        test_size (float): Proportion of data to use for testing
    
    Returns:
        tuple: Prepared training and testing data
    """
    # Convert to numpy array
    X = np.array(X)
    
    # Reshape for CNN-LSTM model
    # Shape will be (samples, height, width, channels)
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    # One-hot encode labels
    y_one_hot = tf.keras.utils.to_categorical(y_encoded)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_one_hot, test_size=test_size, random_state=42
    )
    
    return X_train, X_test, y_train, y_test, label_encoder

In [17]:
def create_cnn_lstm_model(input_shape, num_classes):
    """
    Create CNN-LSTM hybrid model architecture
    
    Args:
        input_shape (tuple): Shape of input data
        num_classes (int): Number of output classes
    
    Returns:
        tf.keras.Model: Compiled model
    """
    model = Sequential([
        # CNN layers for feature extraction
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        # Flatten and reshape for LSTM
        Flatten(),
        
        # LSTM layers for sequence learning
        LSTM(128, return_sequences=True),
        Dropout(0.3),
        LSTM(64),
        Dropout(0.3),
        
        # Dense layers for classification
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    return model

In [18]:
def main():
    # Set base directory
    base_dir = r"C:\Users\New Asus\Documents\FIT4701_2025_Sem1\resources\SHD-HAR-Dataset\SHD-HAR-Dataset-main\amplitude\front"  # MODIFY THIS
    
    # Load data
    X, y = load_activity_data(base_dir)
    
    # Prepare data 
    X_train, X_test, y_train, y_test, label_encoder = prepare_data(X, y)
    
    # Create model
    model = create_cnn_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2], 1), 
        num_classes=y_train.shape[1]
    )
    
    # Print model summary
    model.summary()
    
    # Train model
    history = model.fit(
        X_train, y_train, 
        epochs=100, 
        batch_size=32, 
        validation_split=0.25,
        verbose=1
    )
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
    
    # Optional: Save model and important components
    # model.save('activity_classification_cnn_lstm.h5')
    
    # Save label encoder classes for future reference
    np.save('label_classes.npy', label_encoder.classes_)
    
    # Print class mapping
    print("\nActivity Mapping:")
    for idx, label in enumerate(label_encoder.classes_):
        print(f"{idx}: {label}")
    
    # Optional: Plot training history
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12,4))
    
    # Plot training & validation accuracy values
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot training & validation loss values
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    # plt.savefig('training_history.png')
    plt.close()

In [19]:
if __name__ == "__main__":
    main()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6001,) + inhomogeneous part.